# Money Management Fama-Macbeth Regression Model

This notebook I discuss the famous Fama-Macbeth Regression Model. This analysis assesses the parameters of capital asset pricing model (CAPM), FF-3, and FF-5, to determine the underlying risk premia of the regression parameters. Source is [here](https://github.com/PacktPublishing/Hands-On-Machine-Learning-for-Algorithmic-Trading/blob/master/Chapter07/02_fama_macbeth.ipynb).

This notebook I discuss the following:
- **Environment Initiation**
- **Two-Step Procedure**
- **Fama-Macbeth Cross-Sectional Regression**

## Environment Initiation

Let me import all the required modules.

In [ ]:
from pprint import pprint
from pandas_datareader.famafrench import get_available_datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant
from pathlib import Path
import warnings
from linearmodels.asset_pricing import TradedFactorModel, LinearFactorModel, LinearFactorModelGMM

# due to https://stackoverflow.com/questions/50394873/import-pandas-datareader-gives-importerror-cannot-import-name-is-list-like
# may become obsolete when fixed
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web

We use Fama-French factor model. There was a FF 3-Factor model and a FF 5-Factor model. We introduce the notations below. Each of the concepts is a portfolio, e.g. **PF**. 

| Concept  | Explanation   |
| ------   | ------        |
| Size (SMB)          | Nine small stock PF minus mine large stock PF |
| Value (HML)         | Two value PF minus two growth (with low BE/ME value) PF |
| Profitability (RMW) | Two robust OP PF minus two weak OP PF |
| Investment (CMA)    | Two conservative investment portfolios minus two aggressive investment portfolios |
| Market              | Value-weight return of all firms incorporated in and listed on major US exchanges with good data minus the one-month Treasury bill rate |

In [26]:
ff_factor = 'F-F_Research_Data_5_Factors_2x3'
ff_factor_data = web.DataReader(ff_factor, 'famafrench', start='2010', end='2017-12')[0]
ff_factor_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 6 columns):
Mkt-RF    96 non-null float64
SMB       96 non-null float64
HML       96 non-null float64
RMW       96 non-null float64
CMA       96 non-null float64
RF        96 non-null float64
dtypes: float64(6)
memory usage: 5.2 KB


In [27]:
ff_factor_data.describe()

,Mkt-RF,SMB,HML,RMW,CMA,RF
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,1.158437,0.053854,-0.064896,0.147500,0.049271,0.012604
std,3.579997,2.296482,2.199398,1.550507,1.408406,0.022583
min,-7.890000,-4.550000,-4.500000,-3.990000,-3.330000,0.000000
25%,-0.917500,-1.600000,-1.512500,-1.025000,-0.962500,0.000000
50%,1.235000,0.180000,-0.295000,0.155000,-0.015000,0.000000
75%,3.190000,1.505000,1.140000,1.145000,0.920000,0.010000
max,11.350000,6.810000,8.290000,3.480000,3.700000,0.090000


Fama and French made numerous portfolios that we can illustrate the estimation of the factors exposure. Here there are 17 industry portfolios using monthly data. Let us get these portfolios.

In [28]:
ff_portfolio = '17_Industry_Portfolios'
ff_portfolio_data = web.DataReader(ff_portfolio, 'famafrench', start='2010', end='2017-12')[0]
ff_portfolio_data = ff_portfolio_data.sub(ff_factor_data.RF, axis=0)
ff_factor_data = ff_factor_data.drop('RF', axis=1)
ff_portfolio_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 17 columns):
Food     96 non-null float64
Mines    96 non-null float64
Oil      96 non-null float64
Clths    96 non-null float64
Durbl    96 non-null float64
Chems    96 non-null float64
Cnsum    96 non-null float64
Cnstr    96 non-null float64
Steel    96 non-null float64
FabPr    96 non-null float64
Machn    96 non-null float64
Cars     96 non-null float64
Trans    96 non-null float64
Utils    96 non-null float64
Rtail    96 non-null float64
Finan    96 non-null float64
Other    96 non-null float64
dtypes: float64(17)
memory usage: 13.5 KB


In [29]:
ff_portfolio_data.describe()

,Food,Mines,Oil,Clths,Durbl,Chems,Cnsum,Cnstr,Steel,FabPr,Machn,Cars,Trans,Utils,Rtail,Finan,Other
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,1.045625,0.203229,0.547917,1.396979,1.154896,1.303438,1.136250,1.731250,0.555625,1.351042,1.227604,1.278854,1.465521,0.891250,1.234375,1.243646,1.281979
std,2.795857,7.902683,5.577552,5.025167,5.137095,5.594722,3.174283,5.246562,7.389824,4.694688,4.811242,5.718887,4.151203,3.237306,3.508655,4.808350,3.710443
min,-5.170000,-24.380000,-12.010000,-10.000000,-13.210000,-17.390000,-7.300000,-13.960000,-20.490000,-11.960000,-9.080000,-11.650000,-8.560000,-6.990000,-9.180000,-11.020000,-7.920000
25%,-0.785000,-5.832500,-3.167500,-1.865000,-2.017500,-1.445000,-0.920000,-2.462500,-4.410000,-1.447500,-2.047500,-1.245000,-0.880000,-0.745000,-0.962500,-1.447500,-1.067500
50%,0.930000,-0.415000,1.040000,1.160000,1.205000,1.435000,1.470000,2.190000,0.660000,1.485000,1.545000,0.645000,1.505000,1.215000,0.880000,1.940000,1.575000
75%,3.187500,5.707500,3.915000,3.857500,4.315000,4.442500,3.317500,5.390000,4.220000,3.875000,4.657500,4.802500,4.227500,2.965000,3.355000,4.052500,3.517500
max,6.670000,21.920000,16.300000,17.200000,16.580000,18.370000,8.290000,15.550000,21.350000,17.660000,14.650000,20.860000,13.160000,7.900000,12.360000,13.430000,10.790000


## Two-Step Procedure

Fama-Macbeth regression has a two-step procedure. 

To address the inference problem caused by the correlation of the residuals, Fama and MacBeth proposed a two-step methodology for a cross-sectional regression of returns on factors. The two-stage Fama—Macbeth regression is designed to estimate the premium rewarded for the exposure to a particular risk factor by the market. The two stages consist of:

- **First stage**: N time-series regression, one for each asset or portfolio, of its excess returns on the factors to estimate the factor loadings.

- **Second stage**: T cross-sectional regression, one for each time period, to estimate the risk premium.

In [30]:
betas = []
for industry in ff_portfolio_data:
    step1 = OLS(endog=ff_portfolio_data[industry],
                exog=sm.add_constant(ff_factor_data)).fit()
    betas.append(step1.params.drop('const'))

betas = pd.DataFrame(betas,
                         columns=ff_factor_data.columns,
                         index=ff_portfolio_data.columns)

C:\Users\eagle\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [31]:
betas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, Food  to Other
Data columns (total 5 columns):
Mkt-RF    17 non-null float64
SMB       17 non-null float64
HML       17 non-null float64
RMW       17 non-null float64
CMA       17 non-null float64
dtypes: float64(5)
memory usage: 1.4+ KB


In [32]:
lambdas = []
for period in ff_portfolio_data.index:
    step2 = OLS(endog=ff_portfolio_data.loc[period, betas.index],
                exog=betas).fit()
    lambdas.append(step2.params)

lambdas = pd.DataFrame(lambdas,
                       index=ff_portfolio_data.index,
                       columns=betas.columns.tolist())

In [33]:
lambdas.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 5 columns):
Mkt-RF    96 non-null float64
SMB       96 non-null float64
HML       96 non-null float64
RMW       96 non-null float64
CMA       96 non-null float64
dtypes: float64(5)
memory usage: 10.2 KB


In [34]:
lambdas.mean()

Mkt-RF    1.181043
SMB       0.112553
HML      -1.234931
RMW      -0.341728
CMA      -0.627899
dtype: float64

In [35]:
t = lambdas.mean().div(lambdas.std())
t

Mkt-RF    0.328238
SMB       0.029175
HML      -0.285483
RMW      -0.113031
CMA      -0.181547
dtype: float64

## Fama-Macbeth Model

Let us build Fama-Macbeth Model.

In [36]:
mod = LinearFactorModel(portfolios=ff_portfolio_data, factors=ff_factor_data)
res = mod.fit()
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                 17   R-squared:                      0.6905
No. Factors:                          5   J-statistic:                    17.359
No. Observations:                    96   P-value                         0.1366
Date:                  Wed, Dec 25 2019   Distribution:                 chi2(12)
Time:                          11:34:09                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         1.1810     0.4105     2.8772     0.0040      0.3765      1.9856
SMB            0.1126     0.8869    